# trcFormat Converter
### Johanna Rahm, Sebastian Malkusch
#### Research group Heilemann
#### Institute for Physical and Theoretical Chemistry, Goethe University Frankfurt a.M.
Convert thunderSTORM .csv results table or rapidSTORM localization .txt file to .trc file (seg_id, image_number, x-position [pixel], placeholder, y-position [pixel], intensity). <br/>
The thunderSTORM x/y-positions have to be in nm, intensity in ADU. <br/>
The .trc file is needed for the trackAnalysis JNB and Hidden Markov Modeling. 

## Choose a localization software

In [1]:
from pySPT.widgets import widgetDirectoryStructure
from pySPT.widgets import widgetTrcFormat
from pySPT.analysis import trcFormat
from pySPT.widgets import widgetColumnSort
widget_trc_format = widgetTrcFormat.WidgetTrcFormat()
display(widget_trc_format.software_button)

RadioButtons(options=('thunderSTORM', 'rapidSTORM'), value='thunderSTORM')

## Load file

thunderSTORM: Insert the full file path of the tracked thunderSTORM .csv file or browse for it (Example path: F:\resting\160404_CS5\cell05.tracked.csv) <br/>
rapidSTORM: Insert the full file path of the tracked rapidSTORM .txt file or browse for it. (Example path: Example path: F:\resting\160404_CS5\cell05.tracked.txt)

In [2]:
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()
trc_format = trcFormat.TrcFormat()
display(widget_trc_format.file_text_box, widget_trc_format.file_button)
widget_trc_format.file_button.on_click(widget_trc_format.open_file)
widget_trc_format.file_text_box.observe(widget_trc_format.change_file_box)

TypeError: __init__() missing 6 required positional arguments: 'software', 'file_name', 'pixel_size', 'min_track_type', 'min_track_hmm', and 'seg_id'

## Run analysis

Two new files will be created: (1) track id , frame, x, y, placeholder, intensity, seg id; for the diffusion type analysis (2) track id, frame, x, y, placeholder, intensity; for the hmm analysis. <br/>


In [ ]:
display(widget_trc_format.pixel_size_box)

The min trajectory length for the diffusion type analysis can be applied to the track or the seg id. If applied to the track id, segments with length < min trajectory length can exist. If the further diffusion type analysis is based on segments, applying the filter on seg id is recommended.

In [ ]:
display(widget_trc_format.trajectory_id_button, widget_trc_format.min_track_length_type_box)

The hidden markov modeling is applied to the track id, because state transformations want to be observed. The recommended min track length is 2, to enable the investigation of the entire system.

In [ ]:
display(widget_trc_format.min_track_length_hmm_box)

In [ ]:
display(widget_trc_format.run_button)
def run_analysis(event):
    widget_trc_format.create_clear_output()
    display(widget_trc_format.run_button)
    if widget_trc_format.got_file_name:
        if widget_trc_format.software_button.value == "thunderSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_trc_format.file_text_box.value, widget_trc_format.software_button.value, ['"track.id"' , '"x [nm]"', '"y [nm]"', '"frame"', '"intensity [photon]"', '"seg.id"'])
        elif widget_trc_format.software_button.value == "rapidSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_trc_format.file_text_box.value, widget_trc_format.software_button.value, ['"track.id"', '"Position-0-0"', '"Position-1-0"', '"Amplitude-0-0"', '"ImageNumber-0-0"', '"seg.id"'])
        widget_column_sort.check_header()
        if widget_column_sort.correct_header:
            widget_column_sort.run_column_sort()
            trc_format.trajectory_id = widget_trc_format.trajectory_id_button.value
            trc_format.software = widget_trc_format.software_button.value
            trc_format.column_order = widget_column_sort.column_order
            trc_format.pixel_size = widget_trc_format.pixel_size_box.value
            trc_format.min_track_length_type = widget_trc_format.min_track_length_type_box.value
            trc_format.min_track_length_hmm = widget_trc_format.min_track_length_hmm_box.value
            trc_format.file_name = widget_trc_format.file_text_box.value
            trc_format.load_localization_file()
        else:
            widget_trc_format.warning_wrong_file()
    else:
        widget_trc_format.warning_wrong_file_path()
widget_trc_format.run_button.on_click(run_analysis)

## Save analysis

The .trc file will be saved in the pySPT/analysis folder.

In [ ]:
display(widget_trc_format.save_button)
def save_analysis(event):
    widget_trc_format.create_clear_output()
    display(widget_trc_format.save_button)
    #widget_dir_structure.name_handling(widget_trc_format.file_name)
    widget_dir_structure.name_handling(widget_trc_format.file_text_box.value)
    widget_dir_structure.create_raw_base_name()
    widget_dir_structure.sub_folder = "\\analysis"
    widget_dir_structure.create_folder()
    trc_format.save_trc_file_analysis(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
    widget_dir_structure.sub_folder = "\\hmm"
    widget_dir_structure.create_folder()
    trc_format.save_trc_file_hmm(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
widget_trc_format.save_button.on_click(save_analysis)